In [ ]:
# Import libraries that are required to run your project
# You are allowed to add more libraries as you need

import pandas as pd
import numpy as np
from scipy.stats import spearmanr

Useful to initiate new conda environment given the yml file (Note I adjusted it to work for windows)
```bash
conda env create -f project1_base.yml -n ml4g_project1
```

Note: Maybe we rather make a new one, since it hasn't been updated for 4 years

In [1]:
import os
import zipfile
import shutil

# Define paths
source_dir = 'ML4G_Project_1_Data'  # Path to directory with zip files
target_dir = 'data'  # Target directory for extracted files

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Get all zip files in the source directory
zip_files = [f for f in os.listdir(source_dir) if f.endswith('.zip')]

# Extract each zip file
for zip_file in zip_files:
    zip_path = os.path.join(source_dir, zip_file)
    print(f"Extracting {zip_file}...")
    
    # Create a subfolder named after the zip file (without .zip extension)
    subfolder_name = os.path.splitext(zip_file)[0]
    subfolder_path = os.path.join(target_dir, subfolder_name)
    
    # Remove subfolder if it already exists
    if os.path.exists(subfolder_path):
        shutil.rmtree(subfolder_path)
    
    # Create the subfolder
    os.makedirs(subfolder_path, exist_ok=True)
    
    # Extract zip contents directly to the subfolder
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(subfolder_path)

print(f"All {len(zip_files)} zip files have been extracted to {target_dir}")

Extracting CAGE-train.zip...
Extracting DNase-bed.zip...
Extracting DNase-bigwig.zip...
Extracting H3K27ac-bed.zip...
Extracting H3K27ac-bigwig.zip...
Extracting H3K27me3-bed.zip...
Extracting H3K27me3-bigwig.zip...
Extracting H3K36me3-bed.zip...
Extracting H3K36me3-bigwig.zip...
Extracting H3K4me1-bed.zip...
Extracting H3K4me1-bigwig.zip...
Extracting H3K4me3-bed.zip...
Extracting H3K4me3-bigwig.zip...
Extracting H3K9me3-bed.zip...
Extracting H3K9me3-bigwig.zip...
Extracting sample.zip...
All 16 zip files have been extracted to data


## Work Package 1.1 - Modeling Choices & Data Pre-processing

In [ ]:
# TODO: 
# Load your feature (bed and/or bigwig and/or fasta) and target files (tsv) here.
# Decide which features to use for training. Feel free to process them however you need.

# NOTE: 
# bed and bigwig files contain signals of all chromosomes (including sex chromosomes).
# Training and validation split based on chromosomes has been done for you. 
# However, you can resplit the data in any way you want.

path_data = "/path/to/your/data/files"  # TODO
path_test = "/path/to/test/info/file"   # X3_test_info.tsv ; TODO
test_genes = pd.read_csv(path_test, sep='\t')
# ---------------------------INSERT CODE HERE---------------------------




# ---------------------------------------------------------------------- 

## Work Package 1.2 - Model Building

In [ ]:
# TODO: 
# Select the best model to predict gene expression from the obtained features in WP 1.1.

# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------


## Work Package 1.3 - Prediction on Test Data (Evaluation Metric)

In [ ]:
# TODO:
# Using the model trained in WP 1.2, make predictions on the test data (chr 1 of cell line X3).
# Store predictions in a variable called "pred" which is a numpy array.

pred = None
# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------

# Check if "pred" meets the specified constrains
assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(test_genes), 'Each gene should have a unique predicted expression'

#### Store Predictions in the Required Format

In [ ]:
# Store predictions in a ZIP. 
# Upload this zip on the project website under "Your submission".
# Zip this notebook along with the conda environment (and README, optional) and upload this under "Your code".

save_dir = 'path/to/save/output/file'  # TODO
file_name = 'gex_predicted.csv'         # PLEASE DO NOT CHANGE THIS
zip_name = "LastName_FirstName_Project1.zip" # TODO
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

test_genes['gex_predicted'] = pred.tolist()
test_genes[['gene_name', 'gex_predicted']].to_csv(save_path, compression=compression_options)